# H37Rv blind spots and their attributes

From DOI [10.5281/zenodo.3701839](https://zenodo.org/record/3701840), version 1

In [ ]:
library(tidyverse)

In [ ]:
library(GenomicRanges)

In [ ]:
library(rtracklayer)

### Check the md5 sum. Should be 760937f135f5c463c4e377d8db6cc8ee

In [ ]:
tools::md5sum("S7-seq-attributes.csv") == "760937f135f5c463c4e377d8db6cc8ee"

In [ ]:
h37rv_blind_spots <-  read_csv("S7-seq-attributes.csv",
         col_types = cols(rv_position = col_double(),
              pal = col_skip(),
              homopolymer = col_skip(),
              GCrich = col_skip(),
              repetitive = col_skip(),
              blindspot = col_factor())) %>%
    mutate(chrom = "NC_000962.3")

In [ ]:
head(h37rv_blind_spots)

### Need to compress these individual positions into ranges for the `bed` format

Per https://genome.ucsc.edu/FAQ/FAQformat.html#format1

The first three required BED fields are:

chrom - The name of the chromosome (e.g. chr3, chrY, chr2_random) or scaffold (e.g. scaffold10671).

chromStart - The starting position of the feature in the chromosome or scaffold. The first base in a chromosome is numbered 0.

chromEnd - The ending position of the feature in the chromosome or scaffold. The chromEnd base is not included in the display of the feature, however, the number in position format will be represented. For example, the first 100 bases of chromosome 1 are defined as chrom=1, chromStart=0, chromEnd=100, and span the bases numbered 0-99 in our software (not 0-100), but will represent the position notation chr1:1-100. Read more here.

In [ ]:
h37rv_blind_spots_gr <- makeGRangesFromDataFrame(h37rv_blind_spots %>%
                                                 filter(blindspot == 1),
                         start.field="rv_position",
                         end.field="rv_position")

In [ ]:
head(h37rv_blind_spots_gr)

In [ ]:
width(h37rv_blind_spots_gr)

In [ ]:
length(h37rv_blind_spots_gr)

In [ ]:
range(h37rv_blind_spots_gr)

In [ ]:
(h37rv_blind_spots_gr_reduced <- reduce(h37rv_blind_spots_gr))

In [ ]:
data.frame(seqnames=seqnames(h37rv_blind_spots_gr_reduced),
    starts=start(h37rv_blind_spots_gr_reduced)-1,
    ends=end(h37rv_blind_spots_gr_reduced)) %>%
    write.table(file="h37rv_blind_spots.bed", 
                quote=F, sep="\t", row.names=F, col.names=F)

In [ ]:
head h37rv_blind_spots.bed

# Compare it to Torsten Seeman's mask file

In [ ]:
(torsten_mask_gr <- rtracklayer::import.bed("../TSeeman_H37Rv_mask/Mtb_NC_000962.3_mask.bed"))

In [ ]:
intersect(h37rv_blind_spots_gr_reduced, torsten_mask_gr)

In [ ]:
width(h37rv_blind_spots_gr_reduced) %>% sum

In [ ]:
width(intersect(h37rv_blind_spots_gr_reduced, torsten_mask_gr)) %>% sum

In [ ]:
width(intersect(h37rv_blind_spots_gr_reduced, torsten_mask_gr)) %>% sum / 
width(h37rv_blind_spots_gr_reduced) %>% sum * 100

# Torsten's list only has 46% of the bases in the blind spots paper

In [ ]:
setdiff(h37rv_blind_spots_gr_reduced, torsten_mask_gr)

In [ ]:
width(setdiff(h37rv_blind_spots_gr_reduced, torsten_mask_gr)) %>% sum

In [ ]:
width(setdiff(h37rv_blind_spots_gr_reduced, torsten_mask_gr)) %>% sum / 
width(h37rv_blind_spots_gr_reduced) %>% sum * 100

# As expected, if 46% are covered, 54% of the bases in the blind spots list are independent of Torsten's

In [ ]:
setdiff(torsten_mask_gr, h37rv_blind_spots_gr_reduced)

In [ ]:
width(setdiff(torsten_mask_gr, h37rv_blind_spots_gr_reduced)) %>% sum

In [ ]:
width(setdiff(torsten_mask_gr, h37rv_blind_spots_gr_reduced)) %>% sum / 
width(h37rv_blind_spots_gr_reduced) %>% sum * 100

# Torsten's list has additional bases that equal 85% of the length of the blind spots list

## Might be nice to make some visualizations of the different areas covered by the different lists as well as analyze differences by gene categories